In [6]:
import sys

sys.path.append('../')

from utils.deep_speech import DeepSpeech
from utils.data_processing import load_landmark_openface,compute_crop_radius
from config.config import DINetInferenceOptions
from models.DINet import DINet
from config.config import DINetTrainingOptions

import numpy as np
import glob
import os
import cv2
import torch
import subprocess
import random
from collections import OrderedDict

from models.Gaussian_blur import Gaussian_bluring
from tqdm import tqdm

import cv2
import numpy as np
import torch
import torchlm
from torchlm.tools import faceboxesv2
from torchlm.models import pipnet

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import datetime

# 获取当前时间戳
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [7]:

# set gaussian bluring
g_bluring_whole_image = Gaussian_bluring(radius=3,sigma=4,padding='same')
g_bluring_mouth_region = Gaussian_bluring(radius=5,sigma=100,padding='valid')

def extract_frames_from_video(video_path,save_dir):
    videoCapture = cv2.VideoCapture(video_path)
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    if int(fps) != 25:
        print('warning: the input video is not 25 fps, it would be better to trans it to 25 fps!')
    frames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frame_width = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)
    for i in range(int(frames)):
        ret, frame = videoCapture.read()
        result_path = os.path.join(save_dir, str(i).zfill(6) + '.jpg')
        cv2.imwrite(result_path, frame)
    return (int(frame_width),int(frame_height))

In [8]:
# load config
args = [
    '--source_channel', '3',
    '--mouth_region_size','256',
    '--source_video_path','./template/27-_主播说联播_出_征_东_京_.mp4', # 视频修改此处
    '--driving_audio_path','./template/27-_主播说联播_出_征_东_京_.wav', # 音频修改此处
    '--res_video_dir','./', # 输出视频修改此处
    '--deepspeech_model_path','../asserts/output_graph.pb', # deepspeech 模型修改此处
    '--pretrained_clip_DINet_path', '../asserts/clip_training_DINet_256mouth.pth', # 模型修改此处
]
opt = DINetInferenceOptions().parse_args(args)


if not os.path.exists(opt.source_video_path):
    raise ('wrong video path : {}'.format(opt.source_video_path))

In [9]:

############################################## extract frames from source video ##############################################
print('extracting frames from video: {}'.format(opt.source_video_path))
video_frame_dir = opt.source_video_path.replace('.mp4', '')
if not os.path.exists(video_frame_dir):
    os.mkdir(video_frame_dir)
video_size = extract_frames_from_video(opt.source_video_path,video_frame_dir)

extracting frames from video: ./template/27-_主播说联播_出_征_东_京_.mp4


In [10]:

############################################## extract deep speech feature ##############################################
print('extracting deepspeech feature from : {}'.format(opt.driving_audio_path))
if not os.path.exists(opt.deepspeech_model_path):
    raise ('pls download pretrained model of deepspeech')
DSModel = DeepSpeech(opt.deepspeech_model_path)
if not os.path.exists(opt.driving_audio_path):
    raise ('wrong audio path :{}'.format(opt.driving_audio_path))
ds_feature = DSModel.compute_audio_feature(opt.driving_audio_path)
res_frame_length = ds_feature.shape[0]
ds_feature_padding = np.pad(ds_feature, ((2, 2), (0, 0)), mode='edge')

extracting deepspeech feature from : ./template/27-_主播说联播_出_征_东_京_.wav


In [11]:
############################################## load facial landmark ##############################################
# print('loading facial landmarks from : {}'.format(opt.source_openface_landmark_path))
# if not os.path.exists(opt.source_openface_landmark_path):
#     raise ('wrong facial landmark path :{}'.format(opt.source_openface_landmark_path))
# video_landmark_data = load_landmark_openface(opt.source_openface_landmark_path).astype(np.int64)

# video_landmark_data_origin = load_landmark_openface(opt.source_openface_landmark_path).astype(np.int64)
# print("video_landmark_data.shape:",video_landmark_data_origin.shape)
# print("video_landmark_data[0]:",video_landmark_data_origin[0])  # 都是整数


#######################
## 改为实时生成

cap = cv2.VideoCapture(opt.source_video_path)

# 初始化结果数组
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_landmark_data = np.zeros((num_frames, 68, 2))

# 加载模型
torchlm.runtime.bind(faceboxesv2(device="cuda"))  # set device="cuda" if you want to run with CUDA
# set map_location="cuda" if you want to run with CUDA
torchlm.runtime.bind(
pipnet(backbone="resnet18", pretrained=True,
        num_nb=10, num_lms=68, net_stride=32, input_size=256,
        meanface_type="300w", map_location="cpu", checkpoint=None)
) # will auto download pretrained weights from latest release if pretrained=True

# 逐帧处理视频
frame_index = 0
# 初始化结果数组
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 显示进度条
for frame_index in tqdm(range(num_frames)):
    # print(frame_index,"帧")
    ret, frame = cap.read()
    if not ret:
        break

    # 进行人脸关键点检测
    landmarks, bboxes = torchlm.runtime.forward(frame)

    # 将关键点结果保存到数组中
    for i in range(len(bboxes)):
        video_landmark_data[frame_index] = landmarks[i]

    frame_index += 1

# 释放资源
cap.release()

# 显示结果数组的形状
print("video_landmark_data.shape ",video_landmark_data.shape)  ## 看看跟原来一样不
# print("video_landmark_data[0] ",video_landmark_data[0][0][0])
video_landmark_data = video_landmark_data.astype(np.int64)


## 改为实时生成  ################

/home/dengjunli/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dengjunli/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
 11%|█▏        | 145/1287 [00:24<03:06,  6.11it/s]

In [ ]:

############################################## align frame with driving audio ##############################################
print('aligning frames with driving audio')
video_frame_path_list = glob.glob(os.path.join(video_frame_dir, '*.jpg'))
if len(video_frame_path_list) != video_landmark_data.shape[0]:
    raise ('video frames are misaligned with detected landmarks')
video_frame_path_list.sort()
video_frame_path_list_cycle = video_frame_path_list + video_frame_path_list[::-1]
video_landmark_data_cycle = np.concatenate([video_landmark_data, np.flip(video_landmark_data, 0)], 0)
video_frame_path_list_cycle_length = len(video_frame_path_list_cycle)
if video_frame_path_list_cycle_length >= res_frame_length:
    res_video_frame_path_list = video_frame_path_list_cycle[:res_frame_length]
    res_video_landmark_data = video_landmark_data_cycle[:res_frame_length, :, :]
else:
    divisor = res_frame_length // video_frame_path_list_cycle_length
    remainder = res_frame_length % video_frame_path_list_cycle_length
    res_video_frame_path_list = video_frame_path_list_cycle * divisor + video_frame_path_list_cycle[:remainder]
    res_video_landmark_data = np.concatenate([video_landmark_data_cycle]* divisor + [video_landmark_data_cycle[:remainder, :, :]],0)
res_video_frame_path_list_pad = [video_frame_path_list_cycle[0]] * 2 \
                                + res_video_frame_path_list \
                                + [video_frame_path_list_cycle[-1]] * 2
res_video_landmark_data_pad = np.pad(res_video_landmark_data, ((2, 2), (0, 0), (0, 0)), mode='edge')
assert ds_feature_padding.shape[0] == len(res_video_frame_path_list_pad) == res_video_landmark_data_pad.shape[0]
pad_length = ds_feature_padding.shape[0]

In [ ]:
############################################## randomly select 5 reference images ##############################################
print('selecting five reference images')
ref_img_list = []
resize_w = int(opt.mouth_region_size + opt.mouth_region_size // 4)
resize_h = int((opt.mouth_region_size // 2) * 3 + opt.mouth_region_size // 8)
ref_index_list = random.sample(range(5, len(res_video_frame_path_list_pad) - 2), 5)
for ref_index in tqdm(ref_index_list):
    crop_flag,crop_radius = compute_crop_radius(video_size,res_video_landmark_data_pad[ref_index - 5:ref_index, :, :])
    if not crop_flag:
        raise ('our method can not handle videos with large change of facial size!!')
    crop_radius_1_4 = crop_radius // 4
    ref_img = cv2.imread(res_video_frame_path_list_pad[ref_index- 3])[:, :, ::-1]
    ref_landmark = res_video_landmark_data_pad[ref_index - 3, :, :]
    
    ##### 报错 dengjunli
    
    ref_img_crop = ref_img[
                ref_landmark[29, 1] - crop_radius:ref_landmark[29, 1] + crop_radius * 2 + crop_radius_1_4,
                ref_landmark[33, 0] - crop_radius - crop_radius_1_4:ref_landmark[33, 0] + crop_radius +crop_radius_1_4,
                :]
    
    ## 报错
    
    ref_img_crop = cv2.resize(ref_img_crop,(resize_w,resize_h))
    ref_img_crop = ref_img_crop / 255.0
    ref_img_list.append(ref_img_crop)
ref_video_frame = np.concatenate(ref_img_list, 2)
ref_img_tensor = torch.from_numpy(ref_video_frame).permute(2, 0, 1).unsqueeze(0).float().cuda()

In [ ]:

############################################## load pretrained model weight ##############################################
print('loading pretrained model from: {}'.format(opt.pretrained_clip_DINet_path))
model = DINet(opt.source_channel, opt.ref_channel, opt.audio_channel).cuda()
if not os.path.exists(opt.pretrained_clip_DINet_path):
    raise ('wrong path of pretrained model weight: {}'.format(opt.pretrained_clip_DINet_path))
state_dict = torch.load(opt.pretrained_clip_DINet_path)['state_dict']['net_g']
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]  # remove module.
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()

In [ ]:
print(pre_frame.shape)
print(crop_frame_tensor.shape)
print(crop_frame_tensor)

(416, 320, 3)
torch.Size([1, 3, 416, 320])
tensor([[[[0.8588, 0.8588, 0.8627,  ..., 0.3882, 0.3294, 0.3059],
          [0.8588, 0.8588, 0.8627,  ..., 0.4000, 0.3412, 0.3176],
          [0.8549, 0.8549, 0.8588,  ..., 0.4275, 0.3686, 0.3412],
          ...,
          [0.2157, 0.2157, 0.2196,  ..., 0.1373, 0.1373, 0.1373],
          [0.2157, 0.2157, 0.2196,  ..., 0.1373, 0.1373, 0.1373],
          [0.2157, 0.2157, 0.2196,  ..., 0.1373, 0.1373, 0.1373]],

         [[0.7490, 0.7490, 0.7529,  ..., 0.2902, 0.2392, 0.2157],
          [0.7490, 0.7490, 0.7490,  ..., 0.2980, 0.2471, 0.2235],
          [0.7451, 0.7451, 0.7490,  ..., 0.3255, 0.2706, 0.2471],
          ...,
          [0.2157, 0.2157, 0.2118,  ..., 0.1490, 0.1490, 0.1490],
          [0.2157, 0.2157, 0.2118,  ..., 0.1451, 0.1451, 0.1451],
          [0.2157, 0.2157, 0.2118,  ..., 0.1451, 0.1451, 0.1451]],

         [[0.7059, 0.7059, 0.7059,  ..., 0.2627, 0.2118, 0.1922],
          [0.7059, 0.7059, 0.7059,  ..., 0.2745, 0.2235, 0.2039],

In [ ]:
############################################## inference frame by frame ##############################################
if not os.path.exists(opt.res_video_dir):
    os.mkdir(opt.res_video_dir)
res_video_path = os.path.join(opt.res_video_dir,os.path.basename(opt.source_video_path)[:-4] + '_facial_dubbing.mp4')
if os.path.exists(res_video_path):
    os.remove(res_video_path)
res_face_path = res_video_path.replace('_facial_dubbing.mp4', '_synthetic_face.mp4')
if os.path.exists(res_face_path):
    os.remove(res_face_path)
videowriter = cv2.VideoWriter(res_video_path, cv2.VideoWriter_fourcc(*'XVID'), 25, video_size)
videowriter_face = cv2.VideoWriter(res_face_path, cv2.VideoWriter_fourcc(*'XVID'), 25, (resize_w, resize_h))
for clip_end_index in range(5, pad_length, 1):
    print('synthesizing {}/{} frame'.format(clip_end_index - 5, pad_length - 5))
    crop_flag, crop_radius = compute_crop_radius(video_size,res_video_landmark_data_pad[clip_end_index - 5:clip_end_index, :, :],random_scale = 1.05)
    if not crop_flag:
        raise ('our method can not handle videos with large change of facial size!!')
    crop_radius_1_4 = crop_radius // 4
    frame_data = cv2.imread(res_video_frame_path_list_pad[clip_end_index - 3])[:, :, ::-1]
    frame_landmark = res_video_landmark_data_pad[clip_end_index - 3, :, :]
    crop_frame_data = frame_data[
                        frame_landmark[29, 1] - crop_radius:frame_landmark[29, 1] + crop_radius * 2 + crop_radius_1_4,
                        frame_landmark[33, 0] - crop_radius - crop_radius_1_4:frame_landmark[33, 0] + crop_radius +crop_radius_1_4,
                        :]
    crop_frame_h,crop_frame_w = crop_frame_data.shape[0],crop_frame_data.shape[1]
    crop_frame_data = cv2.resize(crop_frame_data, (resize_w,resize_h))  # [32:224, 32:224, :]
    
    crop_frame_data = crop_frame_data / 255.0
    
    # 取消嘴部区域等于0的操作
    crop_frame_data[opt.mouth_region_size//2:opt.mouth_region_size//2 + opt.mouth_region_size,
                    opt.mouth_region_size//8:opt.mouth_region_size//8 + opt.mouth_region_size, :] = 0

    crop_frame_tensor = torch.from_numpy(crop_frame_data).float().cuda().permute(2, 0, 1).unsqueeze(0)
    deepspeech_tensor = torch.from_numpy(ds_feature_padding[clip_end_index - 5:clip_end_index, :]).permute(1, 0).unsqueeze(0).float().cuda()
    # 或采用自己的deepspeech_tensor

    with torch.no_grad():
        print("检查是否输入正常:",torch.max(crop_frame_tensor),torch.max(ref_img_tensor),torch.max(deepspeech_tensor))
        pre_frame = model(crop_frame_tensor, ref_img_tensor, deepspeech_tensor)
        print(pre_frame.shape)
        print(pre_frame)
        print("zhangliang")
        pre_frame = pre_frame.squeeze(0).permute(1, 2, 0).detach().cpu().numpy() * 255
    videowriter_face.write(pre_frame[:, :, ::-1].copy().astype(np.uint8))
    pre_frame_resize = cv2.resize(pre_frame, (crop_frame_w,crop_frame_h))
    frame_data[
    frame_landmark[29, 1] - crop_radius:
    frame_landmark[29, 1] + crop_radius * 2,
    frame_landmark[33, 0] - crop_radius - crop_radius_1_4:
    frame_landmark[33, 0] + crop_radius + crop_radius_1_4,
    :] = pre_frame_resize[:crop_radius * 3,:,:]
    videowriter.write(frame_data[:, :, ::-1])
videowriter.release()
videowriter_face.release()
video_add_audio_path = res_video_path.replace('.mp4', '_{}_add_audio.mp4'.format(timestamp))
if os.path.exists(video_add_audio_path):
    os.remove(video_add_audio_path)
cmd = 'ffmpeg -i {} -i {} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 {}'.format(
    res_video_path,
    opt.driving_audio_path,
    video_add_audio_path)
subprocess.call(cmd, shell=True)



OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
/home/dengjunli/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


synthesizing 0/1281 frame
torch.Size([1, 3, 416, 320])
tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]]], device='cuda:0',
       dtype

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:3940: error: (-215:Assertion failed) func != 0 in function 'resize'
